# quake 3 server connector



## discord api


Authorize URL:

https://discord.com/oauth2/authorize?response_type=code&client_id=723583889779589221&scope=bot&permissions=2248399936&redirect_uri=https://discord.com/channels/@me



### channel message poller



#### the code

poll discord channel?

discord api?



In [4]:
var fs = require('fs')
var path = require('path')
var {request} = require('gaxios')
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
var credentials
var tokenPath
if(fs.existsSync('./discord-bot.txt')) {
    tokenPath = path.resolve('./discord-bot.txt')
} else {
    tokenPath = path.join(PROFILE_PATH, '.credentials/discord-bot.txt')
}
var token = fs.readFileSync(tokenPath).toString('utf-8').trim()

var DEFAULT_GUILD = process.env.DEFAULT_GUILD || '393252386426191873'
var DEFAULT_CHANNEL = process.env.DEFAULT_CHANNEL || '393252386426191875'
var DEFAULT_APPLICATION = process.env.DEFAULT_APPLICATION || '723583889779589221'
var DEFAULT_API = process.env.DEFAULT_API || 'https://discord.com/api/v8/'
var MESSAGE_TIME = process.env.DEFAULT_TIME || 1000 * 60 * 60 // 1 hour to respond
var DEFAULT_RATE = 2000

var delay = async () => await new Promise(resolve => setTimeout(resolve, DEFAULT_RATE))

async function userGuilds(userId = '@me') {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}users/${userId}/guilds`
    })
    await delay()
    return result.data
}

async function getGuildRoles(guildId = DEFAULT_GUILD) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}guilds/${guildId}/roles`
    })
    await delay()
    return result.data
}

async function userChannels(userId = '@me') {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${userId}`
    })
    await delay()
    return result.data
}

async function userConnections(userId = '@me') {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}users/${userId}/connections`
    })
    await delay()
    return result.data
}

async function guildChannels(guildId = DEFAULT_GUILD) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}guilds/${guildId}/channels`
    })
    await delay()
    return result.data
}

async function channelMessages(channelId = DEFAULT_CHANNEL) {
    var params = {
        limit: 100,
        after: (BigInt(Date.now() - 1420070400000 - MESSAGE_TIME) << BigInt(22)).toString()
    };
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${channelId}/messages`,
        params
    })
    await delay()
    return result.data
}

async function triggerTyping(channelId = DEFAULT_CHANNEL) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'POST',
        url: `${DEFAULT_API}channels/${channelId}/typing`
    })
    await delay()
    return result.data
}

async function createMessage(message, channelId = DEFAULT_CHANNEL) {
    var params = typeof message == 'string' ? ({
        'content': message
    }) : message
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'POST',
        url: `${DEFAULT_API}channels/${channelId}/messages`,
        data: JSON.stringify(params)
    })
    await delay()
    return result.data
}

async function updateMessage(message, messageId, channelId = DEFAULT_CHANNEL) {
    var params = typeof message == 'string' ? ({
        'content': message
    }) : message
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'PATCH',
        url: `${DEFAULT_API}channels/${channelId}/messages/${messageId}`,
        data: JSON.stringify(params)
    })
    await delay()
    return result.data
}

async function registerCommand(cmd, desc, guildId = null) {
    // TODO: guild specific commands
    //url = "https://discord.com/api/v8/applications/<my_application_id>/guilds/<guild_id>/commands"
    var json
    if(typeof cmd == 'object') {
        json = cmd
    } else {
        json = {
            'name': cmd,
            'description': desc,
            'options': []
        }
    }
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'POST',
        url: `${DEFAULT_API}applications/${DEFAULT_APPLICATION}/commands`,
        data: JSON.stringify(json)
    })
    await delay()
    return result.data
}

async function interactionResponse(interactionId, interactionToken) {
    var json = {
        'type': 5
    }
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'POST',
        url: `${DEFAULT_API}interactions/${interactionId}/${interactionToken}/callback`,
        data: JSON.stringify(json)
    })
    await delay()
    return result.data
}

async function getCommands(guildId = null) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}applications/${DEFAULT_APPLICATION}/commands`
    })
    await delay()
    return result.data
}

async function updateInteraction(message, interactionId, interactionToken) {
    var json = typeof message == 'string' ? ({
            'content': message
        }) : message
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'PATCH',
        url: `${DEFAULT_API}webhooks/${DEFAULT_APPLICATION}/${interactionToken}/messages/@original`,
        data: JSON.stringify(json)
    })
    await delay()
    return result.data
}

async function createThread(name, channelId = DEFAULT_CHANNEL) {
    var json = {
        'name': name,
        'type': 11,
        'auto_archive_duration': 60
    }
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`,
            'Content-Type': 'application/json'
        },
        method: 'POST',
        url: `${DEFAULT_API}channels/${channelId}/threads`,
        data: JSON.stringify(json)
    })
    await delay()
    return result.data
}

async function archivedThreads(channelId = DEFAULT_CHANNEL) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${channelId}/threads/archived/public`
    })
    await delay()
    return result.data
}

async function activeThreads(channelId = DEFAULT_CHANNEL) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${channelId}/threads/active`
    })
    await delay()
    return result.data
}

async function deleteCommand(commandId) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'DELETE',
        url: `${DEFAULT_API}applications/${DEFAULT_APPLICATION}/commands/${commandId}`
    })
    await delay()
    return result.data
}

async function getPins(channelId = DEFAULT_CHANNEL) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'GET',
        url: `${DEFAULT_API}channels/${channelId}/pins`
    })
    await delay()
    return result.data
}

async function pinMessage(messageId, channelId = DEFAULT_CHANNEL) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'PUT',
        url: `${DEFAULT_API}channels/${channelId}/pins/${messageId}`
    })
    await delay()
    return result.data
}

async function unpinMessage(messageId, channelId = DEFAULT_CHANNEL) {
    var result = await request({
        headers: {
            Authorization: `Bot ${token}`
        },
        method: 'DELETE',
        url: `${DEFAULT_API}channels/${channelId}/pins/${messageId}`
    })
    await delay()
    return result.data
}



module.exports = {
    userGuilds,
    userChannels,
    userConnections,
    guildChannels,
    getGuildRoles,
    channelMessages,
    triggerTyping,
    createMessage,
    updateMessage,
    registerCommand,
    getCommands,
    interactionResponse,
    updateInteraction,
    createThread,
    archivedThreads,
    activeThreads,
    deleteCommand,
    getPins,
    pinMessage,
    unpinMessage
}


{
  userGuilds: [AsyncFunction: userGuilds],
  guildChannels: [AsyncFunction: guildChannels],
  channelMessages: [AsyncFunction: channelMessages],
  triggerTyping: [AsyncFunction: triggerTyping],
  createMessage: [AsyncFunction: createMessage]
}

#### test sending a discord message?



In [ ]:
var discordApi = importer.import('discord api')
var {authorizeGateway} = importer.import('authorize discord')

async function testMessage()
{
    var discordSocket = await authorizeGateway()
    await discordApi.createMessage('beep boop', '752568660819837019')
    discordSocket.close()
}

module.exports = testMessage


### authorize discord oauth



#### the code

authorize discord?


In [ ]:
var fs = require('fs')
var path = require('path')
var WebSocket = require('ws')
var {request} = require('gaxios')
var importer = require('../Core')
var {interactionResponse} = importer.import('discord api')
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
var tokenPath
if(fs.existsSync('./discord-bot.txt')) {
    tokenPath = path.resolve('./discord-bot.txt')
} else {
    tokenPath = path.join(PROFILE_PATH, '.credentials/discord-bot.txt')
}
var token = fs.readFileSync(tokenPath).toString('utf-8').trim()

var DEFAULT_API = process.env.DEFAULT_API || 'https://discord.com/api/v6/'

var heartbeat
var ws = false
var cancelConnection
var seq = 0
var privateChannels = {}
var interactions = {}

function sendHeartbeat() {
    if(!ws) return
    console.log('Sending heartbeat')
    ws.send(JSON.stringify({
        op: 1,
        d: seq
    }))
    cancelConnection = setTimeout(() => ws ? ws.close() : false, 4000)
}

async function authorizeGateway() {
    if(ws) return
    var result
    try {
        result = await request({
            headers: {
                Authorization: `Bot ${token}`
            },
            method: 'GET',
            url: `${DEFAULT_API}gateway/bot`
        })
    } catch (e) {
        console.log(e.message)
        return
    }
    ws = new WebSocket(result.data.url)
    ws.on('open', () => {
        console.log('Connecting to Discord')
    })
    var identified = false
    ws.on('message', (message) => {
        var msgBuff = new Buffer.from(message)
        var gateway = JSON.parse(msgBuff.toString('utf-8'))
        if(gateway.s) seq = gateway.s
        if(gateway.d && gateway.d.seq) seq = gateway.d.seq
        if(gateway.op == 10) {
            heartbeat = setInterval(sendHeartbeat, gateway.d.heartbeat_interval)
            ws.send(JSON.stringify({
                op: 2,
                intents: ['DIRECT_MESSAGES', 'GUILD_MESSAGES', 'GUILDS'],
                d: {
                    token: token,
                    properties: {
                        "$os": "linux",
                        "$browser": "jupyter",
                        "$device": "quake3"
                    }
                }
            }))
            return
        } else if (gateway.op === 0 || gateway.op === 9) {
            identified = true
            if(gateway.t == 'MESSAGE_CREATE') {
                privateChannels[gateway.d.channel_id] = Date.now()
            }
            if(gateway.t == 'INTERACTION_CREATE') {
                if(typeof interactions[gateway.d.channel_id] == 'undefined')
                    interactions[gateway.d.channel_id] = []
                interactions[gateway.d.channel_id].push(gateway.d)
                interactionResponse(gateway.d.id, gateway.d.token)
            }
            return
        } else if (gateway.op === 11) {
            clearTimeout(cancelConnection)
            return
        }
        console.log(gateway)
    })
    var timer
    ws.on('close', () => {
        console.log('Discord disconnected')
        if(timer) clearInterval(timer)
        clearInterval(heartbeat)
        ws.close()
        ws = false
        return
    })
    await new Promise(resolve => {
        timer = setInterval(() => {
            if(identified) {
                clearInterval(timer)
                resolve()
            }
        }, 1000)
    });
    return ws
}

module.exports = {
    authorizeGateway,
    privateChannels,
    interactions
}


### synchronize discord commands

sync discord commands?


#### the code


In [ ]:
var importer = require('../Core')
var {authorizeGateway} = importer.import('authorize discord')
var {registerCommand, getCommands, deleteCommand} = importer.import('discord api')

async function syncCommands() {
    await authorizeGateway()
    var commandResult = (await getCommands())
    var commands = commandResult.map(command => command.name)
    if(commands.includes('hello-orbb'))
    await deleteCommand(commandResult.filter(c => c.name == 'hello-orbb')[0].id)
    if(!commands.includes('hello'))
    await registerCommand('hello', 'Check if Orbb is awake.')
    if(!commands.includes('challenge'))
    await registerCommand({
        'name': 'challenge',
        'description': 'Challenges another user to match, Orbb waits for the thumbs up.',
        'options': [
            {
                'name': 'opponent-id',
                'description': 'Name of the player you want to challenge for 1v1.',
                'required': true,
                'type': 6
            },
            {
                'name': 'map',
                'description': 'Name of the map to start on the server.',
                'required': true,
                'type': 3
            }
        ]
    })
    if(!commands.includes('connect'))
    await registerCommand({
        'name': 'connect',
        'description': 'RCon Connect to a Quake 3 server for remote administration over Discord.',
        'options': [
            {
                'name': 'server-address',
                'description': 'The IP address or domain name of the server to connect to including port.',
                'required': true,
                'type': 3
            }
        ]
    })
    if(!commands.includes('rcon'))
    await registerCommand({
        'name': 'rcon',
        'description': 'Set the password for future RCon commands, or send an rcon command to the connected server.',
        'options': [
            {
                'name': 'rcon-password',
                'description': 'Password to use with future RCon commands.',
                'required': true,
                'type': 3
            },
            {
                'name': 'rcon-command',
                'description': 'Send the following RCon command to the server.',
                'required': false,
                'type': 3
            }
        ]
    })
    if(!commands.includes('config'))
    await registerCommand({
        'name': 'config',
        'description': 'Execute a config file on the remote Quake 3 server after using /connect command.',
        'options': [
            {
                'name': 'config-name',
                'description': 'Name of the config script to execute',
                'required': true,
                'type': 3
            }
            // TODO: not required and list availabe config scripts through engine
        ]
    })
    if(!commands.includes('map'))
    await registerCommand({
        'name': 'map',
        'description': 'Starts a server with the specified map and sends you a personal message when the server is ready.',
        'options': [
            {
                'name': 'map-name',
                'description': 'Name of the map to run the server.',
                'required': true,
                'type': 3
            }
        ]
    })
    return await getCommands()
}


module.exports = syncCommands


### respond discord commands?

Interesting how this turned out. Commands use the :thumbsup: symbol as a confirmation. Other than that it is `command` followed by `response`. The code tag for `beep boop` contains a hidden language specifier that contains the original commands `message.id`. Because nonce isn't working, it uses a code block.



#### the code



In [ ]:
var importer = require('../Core')
var discordApi = importer.import('discord api')
var {authorizeGateway, privateChannels, interactions} = importer.import('authorize discord')
var {
    discordCommands,
    challengeCommand,
    configCommand,
    connectCommand,
    rconCommand,
    chatCommand,
} = importer.import('discord commands')

var DEFAULT_USERNAME = 'Orbb'

function interpretCommand(message) {
    return Object.keys(discordCommands)
       .filter(k => message.content.match(discordCommands[k])
              || (message.attachments && message.attachments.filter(a => a.filename.match(discordCommands[k])).length > 0)
              || (message.embeds && message.embeds.filter(e => (e.title && e.title.match(discordCommands[k]))
                                                          || (e.description && e.description.match(discordCommands[k]))).length > 0))
}

async function readAllCommands(specificChannel) {
    // matching format  @megamind  challenge freon dm17 , :thumbsup:   :thumbsdown: .
    var private = false
    var messages = []
    var responses = []
    var channels = []
    var commands = []
    var launches = []
    
    if(specificChannel == '@me') {
        // only read channel if it was updated within the last hour
        var userChannels = Object
            .keys(privateChannels)
            .filter(k => privateChannels[k] > Date.now() - 1000 * 60 * 60)
            .map(k => ({id: k}))
        channels.push.apply(channels, userChannels)
        specificChannel = ''
        private = true
    } else {
        var guilds = await discordApi.userGuilds()
        console.log(`Reading ${guilds.length} guilds`)
        for(var i = 0; i < guilds.length; i++) {
            channels.push.apply(channels, await discordApi.guildChannels(guilds[i].id))
        }
    }
    
    console.log(`Reading ${channels.length} channels`)
    for(var i = 0; i < channels.length; i++) {
        if(!specificChannel
           || channels[i].id == specificChannel
           || (typeof specificChannel == 'string'
              && (specificChannel.length === 0
                 || (channels[i].name
                     && channels[i].name.match(new RegExp(specificChannel, 'ig'))
                    )
                 )
              )
          ) {
            console.log(`Reading ${channels[i].name}`)
            messages.push.apply(messages, await discordApi.channelMessages(channels[i].id))
        }
    }
    
    // find commands in channel history
    console.log(`Reading ${messages.length} messages`)
    for(var j = 0; j < messages.length; j++) {
        var applicableCommands = interpretCommand(messages[j])
        if(applicableCommands.length > 0
          && messages[j].author.username != DEFAULT_USERNAME) {
            messages[j].commands = applicableCommands
            messages[j].private = private
            commands.push(messages[j])
            if((messages[j].reactions || [])
                .filter(a => a.emoji.name == '\u{1F44D}').length > 0) {
                launches.push(messages[j])
            }
        }
        if(messages[j].content.match(/```BOT/ig)) {
            responses.push(messages[j])
            if((messages[j].reactions || [])
                .filter(a => a.emoji.name == '\u{1F44D}').length > 0) {
                var l = messages.filter(m => messages[j].content.match('```BOT'+m.id))[0]
                if(!l) continue
                l.launching = true
                l.reactions = l.reactions || []
                l.reactions.push.apply(l.reactions, messages[j].reactions)
                if(l) launches.push(l)
            }
        }
    }
        
    // find all commands in interactions
    var interactionsCount = Object.keys(interactions)
        .reduce((sum, i) => {return sum + interactions[i].length}, 0)
    console.log(`Reading ${Object.keys(interactions).length} channels with ${interactionsCount} interactions`)
    Object.keys(interactions).forEach(i => {
        for(var c in interactions[i]) {
            interactions[i][c].commands = [interactions[i][c].data.name.toUpperCase()]
            interactions[i][c].author = interactions[i][c].member.user
            interactions[i][c].content = interactions[i][c].data.name + ' '
                + (interactions[i][c].data.options || []).map(o => o.value).join(' ')
            interactions[i][c].interaction = true
            commands.push(interactions[i][c])
        }
        interactions[i] = []
    })


    // exclude commands that already got a response
    return commands
        .filter(c => responses.filter(r => r.content.match(new RegExp('```BOT'+c.id))).length === 0)
        .concat(launches)
        .filter(c => responses.filter(r => r.content.match(new RegExp('```BOT'+c.id+'L'))).length === 0)
        .filter((c, i, arr) => arr.indexOf(c) === i)
}

async function respondCommand(specificChannel) {
    await authorizeGateway()
    var commands = await readAllCommands(specificChannel)
    for(var i = 0; i < commands.length; i++) {
        if(commands[i].commands.includes('CHALLENGE'))
            await challengeCommand(commands[i])
        else if(commands[i].commands.includes('CONFIG')) 
            await configCommand(commands[i])
        else if(commands[i].commands.includes('CONNECT'))
            await connectCommand(commands[i])
        else if(commands[i].commands.includes('RCON'))
            await rconCommand(commands[i])
        else if(commands[i].commands.includes('HELLO'))
            await chatCommand(commands[i])
        else if(commands[i].private)
            await unknownCommand(commands[i])
    }
}

module.exports = respondCommand


#### test specific channel?



In [ ]:
var importer = require('../Core')
var respondCommand = importer.import('respond discord commands')

module.exports = function testChannel(channel) {
    respondCommand(channel)
    
}


### challenge discord command?



#### the code



In [ ]:
var importer = require('../Core')
var discordApi = importer.import('discord api')
var serverApi = importer.import('quake 3 server commands')

var CHALLENGE = /(@[^:@\s]+\s*chall?[ae]nge|chall?[ae]nge\s*@[^:@\s]+)\s*([^:@\s]*?)\s*([^:@\s]*?)/ig
var DEFAULT_HOST = process.env.DEFAULT_HOST || 'http://quakeiiiarena.com/play/'
var MODS = typeof process.env.DEFAULT_MODS == 'string'
    ? JSON.parse(process.env.DEFAULT_MODS)
    : [
        'baseq3',
        'freon'
    ]

async function challengeCommand(command) {
    if(!command.private && (!command.mentions || command.mentions.length === 0))
        return
    var options = CHALLENGE.exec(command.content)
    var launch = (options ? options[2] : '') || ''
    var map = (options ? options[3] : '') || ''
    var message = 'I read you'
    var instruction = ''
    if(!MODS.includes(launch) && map.length === 0) {
        map = launch
        launch = ''
    }
    if(map.length === 0) {
        map = 'q3dm17'
    }
    if(launch.length == 0) {
        instruction += ', assuming baseq3 on map ' + map
    } else if(command.launching) {
        instruction += ' ' + launch + ' on map ' + map
    }
    if(!command.launching && !command.content.match(/:thumbsup:/ig)) {
        message = 'Waiting for reaction'
        instruction += ', react with :thumbsup: to launch'
    }
    if(command.launching) {
        message = 'Launching'
        await discordApi.createMessage(message + instruction + '\n```BOT'+command.id+'L\nbeep boop\n```\n', command.channel_id)
        await discordApi.triggerTyping(command.channel_id)
        var masters = await serverApi.listMasters(void 0, void 0, false)
        if(masters.length === 0) {
            await discordApi.createMessage(`Boo hoo, no servers available. :cry:` 
                + '\n```BOT'+command.id+'L\nbeep boop\n```\n', command.channel_id)
            return
        }
        await serverApi.sendRcon(masters[0].ip, masters[0].port, '\exec ' + launch + '.cfg')
        await serverApi.sendRcon(masters[0].ip, masters[0].port, '\map ' + map)
        await new Promise(resolve => setTimeout(resolve, 1000))
        await discordApi.createMessage(`Match is ready ${DEFAULT_HOST}?connect%20${masters[0].ip}:${masters[0].port} (${masters[0].ip}:${masters[0].port})`
                                       + '\n```BOT'+command.id+'L\nbeep boop\n```\n', command.channel_id)
    } else if (instruction.length > 0) {
        await discordApi.createMessage(message + instruction + '\n```BOT'+command.id+'\nbeep boop\n```\n', command.channel_id)
    }
}

module.exports = challengeCommand


### direct messages and discord rcon



#### the code

discord bot?

discord commands?


In [ ]:
var ip6addr = require('ip6addr')
var importer = require('../Core')
var challengeCommand = importer.import('challenge discord command')
var discordApi = importer.import('discord api')
var {
    getInfo, sendRcon, nextInfoResponse,
    nextPrintResponse
} = importer.import('quake 3 server commands')
var formatQuake3Response = importer.import('format quake 3 response')
var removeCtrlChars = importer.import('remove ctrl characters')

var personality = [
    'Yeehaw!',
    'Balls to wall!',
    'Do it to it!',
    'Got it!',
    'Let\'s play!',
    'Roger that!',
    'I read you!',
    'Buenos Dias!'
]

var lose = [
    'Error. Error.',
    'Oops.',
    'Boo hoo!',
    'Phooey!',
    'Au revoir, mon amis.',
    '#*&^@#!!',
]

var discordCommands = {
    CHALLENGE: /^[!\\\/]?(<@[^:@\s]+>\s*chall?[ae]nge|chall?[ae]nge\s*<@[^:@\s]+>)\s*([^:@\s]*?)\s*([^:@\s]*?)/ig,
    CONNECT: /^[!\\\/]?(rcon)?conn?ect\s*([0-9\.a-z-_]+(:[0-9]+)*)$/ig,
    RCON: /^[!\\\/]?rcon(pass?wo?rd)?\s+([^"\s]+)\s*(.*)$/ig,
    DISCONNECT: /[!\\\/]?disconn?ect/ig,
    CONFIG: /^[!\\\/]?(\w*)(\.cfg|config|configure)/ig,
    LOAD: /^[!\\\/]?(load|map)\s*(\w*)/ig,
    COMMAND: /^[!\\\/]/ig,
    HELLO: /^[!\\\/](\w\s*){0,2}hello(\w\s*){0,2}/ig,
    UNKNOWN: /.*/ig,
}

async function configCommand(command) {
    if(!command.attachments && !command.embed) return
    var user = command.author.username
    var options = discordCommands.CONFIG.exec(command.content)
    var options2 = command.attachments
        .map(a => discordCommands.CONFIG.exec(a.filename))
        .filter(a => a)[0]
    var name = options ? options[1] : options2 ? options2[1] : ''
        .replace(options[2], '')
        .replace(options2[2], '')
        .replace(new RegExp(user, 'ig'), '')
        .replace(/[^0-9-_a-z]/ig, '-')
    if(name.length === 0) {
        await discordApi.createMessage(`Couldn't compute filename.` + '\n```BOT'+command.id+'\nbeep boop\n```\n', command.channel_id)
        return
    }
    var file = 'player-' + user + '-' + name + '.cfg'
    await discordApi.triggerTyping(command.channel_id)
    // TODO: remote post
    //await remoteGet(command.attachments[0].url, file, '/home/freonjs/baseq3-cc/conf/')
    await discordApi.createMessage(`exec conf/player-${user}-${name}` + '\n```BOT'+command.id+'\nbeep boop\n```\n', command.channel_id)
}

var userLogins = {}
// username: {address, password, lastUsed, }
async function connectCommand(command) {
    // TODO: record last address and password given
    var user = command.author.username
    var options = discordCommands.CONNECT.exec(command.content)
    if(typeof userLogins[user] == 'undefined')
        userLogins[user] = {}
    userLogins[user] = {
        address: options[2] || userLogins[user].address || 'quakeIIIarena.com',
        password: userLogins[user].password || 'password123!'
    }
    // TODO: try to connect to server and respond with a getinfo print out
    await discordApi.triggerTyping(command.channel_id)
    var match = (/^(.*?):*([0-9]+)*$/ig).exec(userLogins[user].address)
    await getInfo(match[1], parseInt(match[2]) || 27960)
    var info = await nextInfoResponse()
    var filteredKeys = Object.keys(info)
        .filter(k => k != 'challenge'
                && k != 'hostname'
                && k != 'sv_hostname'
                && k != 'mapname'
                && k != 'clients'
                && k != 'g_humanplayers'
                && k != 'sv_maxclients'
                && k != 'ip'
                && k != 'port')
        .map(k => removeCtrlChars(k))
    var filteredValues = filteredKeys
        .map(k => removeCtrlChars(info[k]))
    var json = {
        content: '\n```BOT'+command.id+'\nbeep boop\n```\n',
        embeds: [{
            title: removeCtrlChars(info.sv_hostname || info.hostname || info.gamename || info.game || ''),
            description: info.ip + ':' + info.port,
            color: 0xdda60f,
            fields: [
                {
                    name: 'Map',
                    value: info.mapname,
                    inline: false
                },
                {
                    name: 'Players',
                    value: info.clients + ' (' + (info.g_humanplayers || '?') + ' humans)' + '/' + info.sv_maxclients,
                    inline: false
                },
                {
                    name: 'Key',
                    value: '```http\n' + filteredKeys.join('\n') + '```',
                    inline: true
                },
                {
                    name: 'Value',
                    value: '```yaml\n' + filteredValues.join('\n') + '```',
                    inline: true
                }
            ]
        }]
    }
    
    if(command.interaction)
        await discordApi.updateInteraction(json, command.id, command.token)    
    else
        await discordApi.createMessage(json, command.channel_id)    
}

async function rconCommand(command) {
    var user = command.author.username
    var options = discordCommands.RCON.exec(command.content)
    if(typeof userLogins[user] == 'undefined')
        userLogins[user] = {}
    userLogins[user] = {
        address: userLogins[user].address || 'quakeIIIarena.com',
        password: options[2] || userLogins[user].password || 'password123!'
    }
    await discordApi.triggerTyping(command.channel_id)
    var match = (/^(.*?):*([0-9]+)*$/ig).exec(userLogins[user].address)
    await sendRcon(match[1], parseInt(match[2]) || 27960,
             options[3] && options[3].length > 0
                 ? options[3]
                 : 'cmdlist',
             userLogins[user].password)
    var response = await nextPrintResponse()
    response = formatQuake3Response(response.content, command, response)
    if(typeof response == 'string')
        response += '\n```BOT'+command.id+'\nbeep boop\n```\n'
    else if(typeof response == 'object')
        response.content = '\n```BOT'+command.id+'\nbeep boop\n```\n'
    if(command.interaction)
        await discordApi.updateInteraction(response, command.id, command.token)    
    else
        await discordApi.createMessage(response, command.channel_id)    
}

async function chatCommand(command) {
    if(command.interaction)
        await discordApi.updateInteraction(`Hello.` + '\n```BOT'+command.id+'\nbeep boop\n```\n', command.id, command.token)
    else
        await discordApi.createMessage(`Hello.` + '\n```BOT'+command.id+'\nbeep boop\n```\n', command.channel_id)
    return
}

module.exports = {
    discordCommands,
    challengeCommand,
    configCommand,
    connectCommand,
    rconCommand,
    chatCommand,
}


## quake 3 commands



### quake 3 server commands



#### the code

quake 3 server commands?


In [ ]:
var path = require('path')
var fs = require('fs')
var zlib = require('zlib')
var dgram = require('dgram')
var udpClient = dgram.createSocket('udp4')
var importer = require('../Core')
var {
    getServersResponse, statusResponse, infoResponse
} = importer.import('quake 3 server responses')
var lookupDNS = importer.import('dns lookup')
udpClient.on('message', updateInfo)

var MAX_TIMEOUT = process.env.DEFAULT_TIMEOUT || 10000
var DEFAULT_MASTER = process.env.DEFAULT_MASTER || '207.246.91.235' || '192.168.0.4'
var DEFAULT_PASS = process.env.DEFAULT_PASS || 'password123!'

var masters = []
var nextResponse = {
    nextInfo: null,
    nextStatus: null,
    nextServer: null,
    nextPrint: null,
}

function mergeMaster(master) {
    var found = false
    masters.forEach((ma, i) => {
        if(ma['ip'] == master['ip'] && ma['port'] == master['port']) {
            found = true
            Object.assign(masters[i], master)
            Object.assign(master, masters[i])
            return false
        }
    })
    if(!found)
        masters.push(master)
    return Object.assign({}, master)
}

async function updateInfo(m, rinfo) {
    if(m[0] == 255 && m[1] == 255 && m[2] == 255 && m[3] == 255)
        m = m.slice(4, m.length)

    if(m.slice(0, 'getserversResponse'.length).toString('utf-8').toLowerCase() == 'getserversresponse'
      || m.slice(0, 'getserversExtResponse'.length).toString('utf-8').toLowerCase() == 'getserversextresponse') {
        var masters = getServersResponse(m)
        for(var m in masters) {
            mergeMaster(masters[m])
            await getStatus(masters[m].ip, masters[m].port)
        }
        nextResponse.nextServer = mas
    } else if (m.slice(0, 'statusResponse'.length).toString('utf-8').toLowerCase() == 'statusresponse') {
        var status = statusResponse(m)
        mergeMaster(Object.assign(status, {
            ip: rinfo.address,
            port: rinfo.port
        }))
        nextResponse.nextStatus = status
    } else if (m.slice(0, 'infoResponse'.length).toString('utf-8').toLowerCase() == 'inforesponse') {
        var info = infoResponse(m)
        mergeMaster(Object.assign(info, {
            ip: rinfo.address,
            port: rinfo.port
        }))
        nextResponse.nextInfo = info
    } else if (m.slice(0, 'print'.length).toString('utf-8') == 'print') {
        var print = {
            content: m.slice('print'.length).toString('utf-8')
        }
        print = Object.assign(mergeMaster({
            ip: rinfo.address,
            port: rinfo.port
        }), print)
        nextResponse.nextPrint = print
    } else if (m.slice(0, 'challengeResponse'.length).toString('utf-8').toLowerCase() == 'challengeresponse') {
        var challenge = mergeMaster(Object.assign({
            challenge: m.slice('challengeResponse'.length).toString('utf-8').trim().split(/\s+/ig)[0]
        }, {
            ip: rinfo.address,
            port: rinfo.port
        }))
        nextResponse.nextChallenge = challenge
    } else if (m.slice(0, 'connectResponse'.length).toString('utf-8').toLowerCase() == 'connectresponse') {
        var challenge = mergeMaster(Object.assign({
            challenge: m.slice('connectResponse'.length).toString('utf-8').trim().split(/\s+/ig)[0]
        }, {
            ip: rinfo.address,
            port: rinfo.port
        }))
        nextResponse.nextConnect = challenge
    } else {
    }
    nextResponse.nextAny = {
        content: m.toString('utf-8')
    }
    Object.assign(nextResponse.nextAny, {
        ip: rinfo.address,
        port: rinfo.port
    })
    if(!nextResponse.nextAll) {
        nextResponse.nextAll = []
    }
    nextResponse.nextAll.push(nextResponse.nextAny)
}

async function getNextResponse(key) {
    var timeout = 0
    nextResponse[key] = null
    return new Promise(resolve => {
        var waiter
        waiter = setInterval(() => {
            timeout += 20
            if(nextResponse[key] != null || timeout >= MAX_TIMEOUT) {
                clearInterval(waiter)
                resolve(nextResponse[key])
            }
        }, 20)
    })
}

async function nextInfoResponse() {
    return await getNextResponse('nextInfo')
}

async function nextServerResponse() {
    return await getNextResponse('nextServer')
}

async function nextStatusResponse() {
    return await getNextResponse('nextStatus')
}

async function nextPrintResponse() {
    return await getNextResponse('nextPrint')
}

async function nextChallengeResponse() {
    return await getNextResponse('nextChallenge')
}

async function nextConnectResponse() {
    return await getNextResponse('nextConnect')
}

async function nextAnyResponse() {
    return await getNextResponse('nextAny')
}

async function nextAllResponses() {
    return await getNextResponse('nextAll')
}

async function getChallenge(address, port = 27960, challenge, gamename) {
    var dstIP = await lookupDNS(address)
    var msgBuff = new Buffer.from(`\xFF\xFF\xFF\xFFgetchallenge ${challenge} ${gamename}`.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
}

async function sendConnect(address, port = 27960, info) {
    var connectInfo = typeof info == 'string' 
        ? info 
        : Object.keys(info).map(k => '\\' + k + '\\' + info[k])
    var dstIP = await lookupDNS(address)
    var msgBuff = new Buffer.from(`\xFF\xFF\xFF\xFFconnect "${connectInfo}"`.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
}

async function sendRcon(address, port = 27960, command, password = DEFAULT_PASS) {
    var dstIP = await lookupDNS(address)
    var msgBuff = new Buffer.from(`\xFF\xFF\xFF\xFFrcon "${password}" ${command}`.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
}

async function getStatus(address, port = 27960) {
    var dstIP = await lookupDNS(address)
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetstatus'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
}

async function getInfo(address, port = 27960) {
    var dstIP = await lookupDNS(address)
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetinfo xxx'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
}

async function listMasters(master = DEFAULT_MASTER, port = 27950, wait = true) {
    var dstIP = await lookupDNS(master)
    var msgBuff = new Buffer.from('\xFF\xFF\xFF\xFFgetservers 68 empty'.split('').map(c => c.charCodeAt(0)))
    udpClient.send(msgBuff, 0, msgBuff.length, port, dstIP)
    if(wait) {
        await new Promise(resolve => setTimeout(resolve, MAX_TIMEOUT))
    } else {
        var timeout = 0
        var timer
        // can't use nextInfoResponse() because it depends on at least 1 statusResponse
        await nextStatusResponse()
    }
    return masters
}

module.exports = {
    listMasters,
    getInfo,
    getStatus,
    getChallenge,
    sendRcon,
    udpClient,
    nextInfoResponse,
    nextStatusResponse,
    nextServerResponse,
    nextPrintResponse,
    nextChallengeResponse,
    nextAnyResponse,
    nextAllResponses,
}


#### test quake 3 rcon commands?



In [ ]:
var importer = require('../Core')
var serverApi = importer.import('quake 3 server commands')

async function testRcon (command) {
    var masters = await serverApi.listMasters(void 0, void 0, false)
    console.log(masters)
    await serverApi.sendRcon(masters[0].ip, masters[0].port, command)
    await new Promise(resolve => setTimeout(resolve, 1000))
    serverApi.udpClient.close()
}

module.exports = testRcon


### quake 3 server responses



#### the code

quake 3 server responses?



In [ ]:

function getServersResponse(m) {
    var masters = []
    m = m.slice('getserversResponse'.length)
    for(var i = 0; i < m.length / 7; i++) {
        var ip = i*7+1
        if(m[ip-1] !== '\\'.charCodeAt(0)) continue
        if(m.slice(ip, ip+3) == 'EOT') continue
        var master = {
            ip: m[ip] + '.' + m[ip+1] + '.' + m[ip+2] + '.' + m[ip+3],
            port: (m[ip+4] << 8) + m[ip+5],
        }
        masters.push(master)
    }
    return masters
}

function parseConfigStr(m) {
    return m.toString('utf-8')
        .trim().split(/\n/ig)[0]
        .trim().split(/\\/ig).slice(1)
        .reduce((obj, c, i, arr) => {
            if(i & 1) obj[arr[i-1]] = c
            return obj
        }, {})
}

function statusResponse(m) {
    m = m.slice('statusResponse'.length)
    var status = parseConfigStr(m)
    var players = m.toString('utf-8')
        .trim().split(/\n/ig)
        .slice(1)
        .reduce((obj, c, i, arr) => {
            if(c.trim().length == 0)
                return obj
            var player = {
                'i': (i + 1),
                'name': (/[0-9]+\s+[0-9]+\s+"(.*)"/ig).exec(c)[1],
                'score': (/([0-9]+)\s+/ig).exec(c)[1],
                'ping': (/[0-9]+\s+([0-9]+)\s+/ig).exec(c)[1],
            }
            player.isBot = player.ping == 0
            obj.push(player)
            return obj
        }, [])
    return Object.assign(status, {players})
}

function infoResponse(m) {
    m = m.slice('infoResponse'.length)
    return parseConfigStr(m)
}

module.exports = {
    getServersResponse,
    statusResponse,
    infoResponse,
    parseConfigStr,
}


### format quake 3 response?



#### the code



In [ ]:
var removeCtrlChars = importer.import('remove ctrl characters')
var importer = require('../Core')

function formatQuake3Response(response, command, server) {
    // try to detect format from response
    var map = (/map:\s(.+)$/igm).exec(response)
    var status = response.match(/name/ig) && response.match(/ping/ig)
    var div = (/^[\-\s]+$/igm).exec(response)
    var players = importer.regexToArray(/^\s*([0-9]+)\s+([0-9]+)\s+([0-9]+)\s+([^\s]+)\s+([^\s]+)\s+.*?$/igm, response, false)
    if(map && status && div) {
        server.mapname = map[1]
        return {
            embeds: [{
                title: removeCtrlChars(server.sv_hostname || server.hostname || server.gamename || server.game || ''),
                description: server.ip + ':' + server.port,
                color: 0xdda60f,
                fields: [
                    {
                        name: 'Map',
                        value: map[1],
                        inline: false
                    },
                    {
                        name: 'Players',
                        value: server.clients + '/' + server.sv_maxclients,
                        inline: false
                    },
                    {
                        name: 'Player',
                        value: '```http\n' + players.map((p, i) => i + ') ' + removeCtrlChars(p[4])).join('\n') + '```',
                        inline: true
                    },
                    {
                        name: 'Score',
                        value: '```yaml\n' + players.map(p => p[2]).join('\n') + '```',
                        inline: true
                    },
                    {
                        name: 'Ping',
                        value: '```fix\n' + players.map(p => p[3]).join('\n') + '```',
                        inline: true
                    }
                ]
            }]
        }
    }
    return '\n```\n' + response + '\n```\n'
}

module.exports = formatQuake3Response


## utilities



### Huffman decoder



#### the code

huffman decode?


In [ ]:
var fs = require('fs')
var huffman = fs.readFileSync('/Users/briancullinan/planet_quake/build/debug-js-js/huffman_js.wasm')
//var Huffman = require('/Users/briancullinan/planet_quake/code/xquakejs/lib/huffman.js')

var MAX_STRING_CHARS = 1024
var buffer
Huffman.onRuntimeInitialized = () => {
    Huffman['_MSG_initHuffman']()
    buffer = Huffman.allocate(new Int8Array(MAX_STRING_CHARS), 'i8', 0)
}


function readBits(m, offset, bits = 8) {
    var value
    var nbits = bits & 7
    var sym = Huffman.allocate(new Int32Array(1), 'i32', 1)
    var bitIndex = offset
    m.forEach((c,i) => Huffman.HEAP8[buffer+i] = c)
    if ( nbits )
    {
        for ( i = 0; i < nbits; i++ ) {
            value |= Huffman._HuffmanGetBit( buffer, bitIndex ) << i
            bitIndex++
        }
        bits -= nbits
    }
    if ( bits )
    {
        for ( i = 0; i < bits; i += 8 )
        {
            bitIndex += Huffman._HuffmanGetSymbol( sym, buffer, bitIndex )
            value |= ( Huffman.getValue(sym) << (i+nbits) )
        }
    }
    return value
}

function decompressMessage(message, offset) {
	message.forEach((c,i) => Huffman.HEAP8[msgData+i] = c)
	Huffman.HEAP32[(msg>>2)+5] = message.length
	Huffman._Huff_Decompress( msg, 12 )
	return Huffman.HEAP8.slice(msgData + offset, msgData + Huffman.HEAP32[(msg>>2)+5])
}

async function init() {
  var binary = new Uint8Array(huffman)
  let imports = {};
  imports['memory'] = new WebAssembly['Memory']( {'initial': 32} )
  memory = new Uint8Array( imports['memory']['buffer'] )
  let program = await WebAssembly.instantiate(binary, { env: imports })
  console.log(program.instance.exports)
}

init()

module.exports = {
    readBits,
    decompressMessage
}


### monitor q3 servers?

Idempotent function to manage discord threads based on player activity on Q3 servers

#### the code


In [ ]:
var importer = require('../Core')
var serverApi = importer.import('quake 3 server commands')
var {authorizeGateway} = importer.import('authorize discord')
var discordApi = importer.import('discord api')
var removeCtrlChars = importer.import('remove ctrl characters')
var DEFAULT_USERNAME = 'Orbb'

var monitorRunning = false
async function monitorServers() {
    if(monitorRunning)
        return
    monitorRunning = true
    /*
    // TODO: search a specific master server for servers with players
    var servers = await serverApi.listMasters('master.ioquake3.org', void 0, true)
    servers.forEach(server => {
        console.log(server)
    })
    */

    //await serverApi.getStatus('quakeIIIarena.com', 5000)
    await serverApi.getStatus('q3msk.ru', 27977)
    var server = await serverApi.nextStatusResponse()
    var threadName = 'Pickup for ' + removeCtrlChars(server.sv_hostname || server.hostname).replace(/[^0-9a-z-]/ig, '-')
    var redTeam = (server.Players_Red || '').split(/\s+/ig).map(i => parseInt(i))
    var blueTeam = (server.Players_Blue || '').split(/\s+/ig).map(i => parseInt(i))
    var json = {
        embeds: [{
            title: removeCtrlChars(server.sv_hostname || server.hostname || server.gamename || server.game || ''),
            description: server.ip + ':' + server.port,
            color: 0xdda60f,
            fields: [
                {
                    name: 'Map',
                    value: server.mapname,
                    inline: false
                },
                {
                    name: 'Players',
                    value: server.players.length + '/' + server.sv_maxclients,
                    inline: false
                },
                {
                    name: 'Player',
                    value: '```http\n' + server.players.map((p, i) => (p.i) + ') ' 
                        + (redTeam.includes(p.i) ? ':red_circle: ' : (blueTeam.includes(p.i) ? ':blue_circle: ' : ''))
                        + removeCtrlChars(p.name)).join('\n') + '```',
                    inline: true
                },
                {
                    name: 'Score',
                    value: '```yaml\n' + server.players.map(p => p.score).join('\n') + '```',
                    inline: true
                },
                {
                    name: 'Ping',
                    value: '```fix\n' + server.players.map(p => p.ping).join('\n') + '```',
                    inline: true
                }
            ]
        }]
    }

    // TODO: comment this line out when launched from index? monitor script
    await authorizeGateway()

    var archived = (await discordApi.archivedThreads()).threads // TODO: sort ffa/ctf/freeZe
        .filter(t => t.name == threadName)

    var thread
    if(archived.length > 0) {
        thread = archived[0]
        var pins = (await discordApi.getPins(thread.id))
            .filter(p => p.author.username == DEFAULT_USERNAME)
        if(pins.length > 0)
            await discordApi.unpinMessage(pins[0], thread.id)
    } else {
        var active = (await discordApi.activeThreads()).threads
            .filter(t => t.name == threadName)
        if(active.length > 0) {
            // find and update previous "whos online" message, pins?
            thread = active[0]
            var pins = (await discordApi.getPins(thread.id))
                .filter(p => p.author.username == DEFAULT_USERNAME)
            if(pins.length > 0) {
                await discordApi.updateMessage(json, pins[0].id, thread.id)

                monitorRunning = false
                return
            }
        } else {
            thread = await discordApi.createThread(threadName)
        }
    }
    // create new "whos online message"
    var message = await discordApi.createMessage(json, thread.id)
    await discordApi.pinMessage(message.id, thread.id)

    monitorRunning = false
}

module.exports = monitorServers


### cron entry



#### the code

index?

discord bot index?


In [ ]:
var importer = require('../Core')
var respondCommand = importer.import('respond discord commands')
var monitorServers = importer.import('monitor q3 servers')

var DEFAULT_CHANNEL = process.env.DEFAULT_CHANNEL || 'general'

var stillRunning = false
async function startResponder() {
    if(stillRunning) {
        console.log('Still running...')
        return
    }
    stillRunning = true
    try {
        await respondCommand(DEFAULT_CHANNEL)
        await respondCommand('@me')
    } catch (e) {
        console.log(e)
    }
    stillRunning = false
}
setInterval(startResponder, 5000)
//setInterval(monitorServers, 5000)

module.exports = startResponder


### ssh curl command

Run the curl command on a remote server to download files from discord and maps from ws.q3df.org.



#### the code

ssh remote wget?



In [ ]:
var fs = require('fs')
var path = require('path')
var NodeSSH = require('node-ssh')
var ssh = new NodeSSH()
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
var privateKeyPath
if(fs.existsSync('./id_rsa')) {
    privateKeyPath = path.resolve('./id_rsa')
} else {
    privateKeyPath = path.join(PROFILE_PATH, '.ssh/id_rsa')
}

var DEFAULT_SSH = process.env.DEFAULT_SSH || 'okayfun.com'
var DEFAULT_SSH_USER = process.env.DEFAULT_SSH_USER || 'root'

/*
ssh.connect({
  host: DEFAULT_SSH,
  username: DEFAULT_SSH_USER,
  privateKey: privateKeyPath
})
*/

async function remoteGet(url, output, cwd) {
    var options = {
        cwd: cwd,
        onStdout(chunk) {
          console.log('stdoutChunk', chunk.toString('utf8'))
        },
        onStderr(chunk) {
          console.log('stderrChunk', chunk.toString('utf8'))
        },
    }
    try {
        await ssh.exec('/usr/bin/wget', ['-O', output, url], options)
        await ssh.exec(`
fileLength=$(wc -l ../index.json | cut -d' ' -f1);
sed "$((fileLength-1))s/$/,/;
${fileLength}i  \\\t\"\":\"\"" ../index.json`, [], options)
        
    } catch (e) {
        console.log(e)
    }
}

module.exports = remoteGet


### dns lookup?



#### the code



In [ ]:
var dns = require('dns')
var _dnsLookup = {}

async function lookupDNS(address) {
  if(typeof _dnsLookup[address] != 'undefined')
    return _dnsLookup[address]
  return new Promise((resolve, reject) => dns.lookup(address, function(err, dstIP) {
    if(err) {
      return reject(err)
    }
    _dnsLookup[address] = dstIP
    return resolve(dstIP)
  }))
}

module.exports = lookupDNS


### remove ctrl characters?



#### the code



In [ ]:

// TODO: use this for server names and player names when matching to discord
function removeCtrlChars(str) {
    return str
        .replace(/\^\^[a-z0-9][a-z0-9]/ig, '')
        .replace(/\^[a-z0-9]/ig, '')
        .trim()
}

module.exports = removeCtrlChars


### get server status with gamedig

quake3 server status?



#### the code



In [ ]:
var importer = require('../Core')
var gamedig = require('gamedig')
var serverApi = importer.import('quake 3 server commands')
var { sendRcon, nextAllResponses, udpClient } = importer.import('quake 3 server commands')
var discordApi = importer.import('discord api')
var {authorizeGateway} = importer.import('authorize discord')
var {parseConfigStr} = importer.import('quake 3 server responses')
var removeCtrlChars = importer.import('remove ctrl characters')

async function getStatus(ip, port) {
    return gamedig.query({
        type: 'quake3',
        host: ip,
        port: port
    }).then((state) => {
        return state
    }).catch((error) => {
        console.log('Server is offline', error)
    })
}

async function captureAllStats() {
    var masters = await serverApi.listMasters('master.ioquake3.org', void 0, false)
    //var status = await getStatus(masters[1].ip, masters[1].port)
    var status = await getStatus('45.32.237.139', 27960)
    console.log(status.bots)
}

//typedef enum {
var SV_EVENT = {
	MAPCHANGE: 0,
    CLIENTSAY: 1,
    MATCHEND: 2,
    CALLADMIN: 3,
    CLIENTDIED: 4,
    CLIENTWEAPON: 5,
    CLIENTRESPAWN: 6,
    CLIENTAWARD: 7,
    GETSTATUS: 8,
    SERVERINFO: 9,
    CONNECTED: 10,
    DISCONNECT: 11,
}
//} recentEvent_t;


async function getChats(channelId) {
    var match = (/^(.*?):*([0-9]+)*$/ig).exec()
    await sendRcon('127.0.0.1', 27960, '', 'recentPassword')
    var response = await nextAllResponses()

    if(!response) return

    var maxTime = 0
    var parsed = response.map(function (r) {
        return JSON.parse(r.content)
    })
    var chats = parsed.filter(function (r) {
        if(r.timestamp > maxTime)
            maxTime = r.timestamp
        return r.type == SV_EVENT.CLIENTSAY
    })
    
    var call = parsed.filter(function (r) {
        return r.type == SV_EVENT.CALLADMIN
    })
    
    var status = parsed.filter(function (r) {
        return r.type == SV_EVENT.GETSTATUS
    })
    var server = {}
    if(status.length) {
        Object.assign(server, parseConfigStr(status[0].value))
    }

    var info = parsed.filter(function (r) {
        return r.type == SV_EVENT.SERVERINFO
    })
    if(info.length) {
        Object.assign(server, parseConfigStr(info[0].value))
    }
    
    var match = parsed.filter(function (r) {
        return r.type == SV_EVENT.MATCHEND
    })
    if(match.length) {
        // TODO: save to SQL database
        console.log(match[match.length-1])
    }

    var discordSocket = await authorizeGateway()
    //console.log(await discordApi.getGuildRoles('752561748611039362'))
    if(call.length) {
        await discordApi.triggerTyping(channelId)        
    }
    for(var i = 0; i < call.length; i++) {
        try {
            //console.log('Say: ' + call[i].value)
            await discordApi.createMessage({
                embed: {
                    title: removeCtrlChars(server.hostname || server.sv_hostname || server.gamename),
                    description: server.ip + ':' + server.port,
                    color: 0xdda60f,
                    fields: [
                        {
                            name: call[i].value,
                            value: `<@&752605581029802155> [Connect](https://quake.games/?connect%20${'127.0.0.1:27960'})`,
                            inline: false
                        },
                    ]
                },
                allowed_mentions: {
                    parse: ['users', 'roles'],
                    users: [],
                    roles: []
                }
            }, channelId)
            //await discordApi.createMessage(`@admin ${call[i].value}`, channelId)
        } catch (e) {
            console.log(e)
        }
    }
}

module.exports = getChats

### spectate q3 server?

Simulate a full player connection to spectate a quake 3 match. Stays connected, switches maps, monitors for chat messages.


#### the code


In [ ]:
var importer = require('../Core')
var {
    getInfo, nextInfoResponse,
    getChallenge, nextChallengeResponse,
    udpClient
} = importer.import('quake 3 server commands')

async function spectateServer(address = 'okayfun.com', port = 27960) {
    var challenge = new ArrayBuffer(4)
    for(var c = 0; c < 4; c++) {
        challenge[c] = Math.round(Math.random() * 255)
    }
    await getInfo(address, port)
    var info = await nextInfoResponse()
    await getChallenge(address, port, new Uint32Array(challenge)[0], info.gamename || info.game)
    var challenge = await nextChallengeResponse()
    await sendConnect(address, port, {
        qport: socket.address().port,
        challenge: challenge,
        protocol: 68,
        
    })
    challenge = await nextConnectResponse()
    
    // TODO: begin netchan compression
    // TODO: await gamestate
    // TODO: await and ACK snapshots
    // TODO: await print commands
    // TODO: forward print commands to discord
}


module.exports = spectateServer
